In [153]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import xgboost as xgb 
from xgboost import XGBClassifier
from collections import Counter

In [154]:
train_df = pd.read_csv("../data/train.csv")
test_df = pd.read_csv("../data/test.csv")
data_df = pd.concat([train_df,test_df])

In [155]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 132.9+ KB


In [156]:
data_df['Title'] = ''

# Cleaning name and extracting Title
for name_string in data_df['Name']:
    data_df['Title'] = data_df['Name'].str.extract('([A-Za-z]+)\.', expand=True)

# Replacing rare titles with more common ones
mapping = {'Mlle': 'Miss', 'Major': 'Mr', 'Col': 'Mr', 'Sir': 'Mr', 'Don': 'Mr', 'Mme': 'Miss',
          'Jonkheer': 'Mr', 'Lady': 'Mrs', 'Capt': 'Mr', 'Countess': 'Mrs', 'Ms': 'Miss', 'Dona': 'Mrs'}

data_df.replace({'Title': mapping}, inplace=True)
titles = ['Dr', 'Master', 'Miss', 'Mr', 'Mrs', 'Rev']

for title in titles:
    age_to_impute = data_df.groupby('Title')['Age'].median()[titles.index(title)]
    data_df.loc[(data_df['Age'].isnull()) & (data_df['Title'] == title), 'Age'] = age_to_impute
    
# Substituting Age values in TRAIN_DF and TEST_DF:
train_df['Age'] = data_df['Age'][:891]
test_df['Age'] = data_df['Age'][891:]

# Dropping Title feature
data_df.drop('Title', axis = 1, inplace = True)

In [157]:
data_df.Age.isna().sum()

0

In [158]:
data_df['Family_Size'] = data_df['Parch'] + data_df['SibSp']

# Substituting Age values in TRAIN_DF and TEST_DF:
train_df['Family_Size'] = data_df['Family_Size'][:891]
test_df['Family_Size'] = data_df['Family_Size'][891:]

In [159]:
0
data_df['Lastname'] = data_df['Name'].apply(lambda x: str.split(x, ",")[0])
data_df['Fare'].fillna(data_df['Fare'].mean(), inplace=True)

DEFAULT_SURVIVAL_VALUE = 0.5
data_df['Family_Survival'] = DEFAULT_SURVIVAL_VALUE

for grp, grp_df in data_df[['Survived','Name', 'Lastname', 'Fare', 'Ticket', 'PassengerId',
                           'SibSp', 'Parch', 'Age', 'Cabin']].groupby(['Lastname', 'Fare']):
    
    if (len(grp_df) != 1):
        # A Family group is found.
        for ind, row in grp_df.iterrows():
            smax = grp_df.drop(ind)['Survived'].max()
            smin = grp_df.drop(ind)['Survived'].min()
            passID = row['PassengerId']
            if (smax == 1.0):
                data_df.loc[data_df['PassengerId'] == passID, 'Family_Survival'] = 1
            elif (smin==0.0):
                data_df.loc[data_df['PassengerId'] == passID, 'Family_Survival'] = 0

print("Number of passengers with family survival information:", 
      data_df.loc[data_df['Family_Survival']!=0.5].shape[0])

Number of passengers with family survival information: 420


In [160]:
for _, grp_df in data_df.groupby('Ticket'):
    if (len(grp_df) != 1):
        for ind, row in grp_df.iterrows():
            if (row['Family_Survival'] == 0) | (row['Family_Survival']== 0.5):
                smax = grp_df.drop(ind)['Survived'].max()
                smin = grp_df.drop(ind)['Survived'].min()
                passID = row['PassengerId']
                if (smax == 1.0):
                    data_df.loc[data_df['PassengerId'] == passID, 'Family_Survival'] = 1
                elif (smin==0.0):
                    data_df.loc[data_df['PassengerId'] == passID, 'Family_Survival'] = 0
                        
print("Number of passenger with family/group survival information: " 
      +str(data_df[data_df['Family_Survival']!=0.5].shape[0]))

# # Family_Survival in TRAIN_DF and TEST_DF:
train_df['Family_Survival'] = data_df['Family_Survival'][:891]
test_df['Family_Survival'] = data_df['Family_Survival'][891:]

Number of passenger with family/group survival information: 546


In [161]:
data_df['Fare'].fillna(data_df['Fare'].median(), inplace = True)

# Making Bins
data_df['FareBin'] = pd.qcut(data_df['Fare'], 5)#binning

label = LabelEncoder()
data_df['FareBin_Code'] = label.fit_transform(data_df['FareBin'])

train_df['FareBin_Code'] = data_df['FareBin_Code'][:891]
test_df['FareBin_Code'] = data_df['FareBin_Code'][891:]

train_df.drop(columns='Fare', inplace=True)
test_df.drop(columns='Fare', inplace=True)

In [162]:
data_df['AgeBin'] = pd.qcut(data_df['Age'], 4)#binning

label = LabelEncoder()
data_df['AgeBin_Code'] = label.fit_transform(data_df['AgeBin'])

train_df['AgeBin_Code'] = data_df['AgeBin_Code'][:891]
test_df['AgeBin_Code'] = data_df['AgeBin_Code'][891:]

train_df.drop(columns='Age', inplace=True)
test_df.drop(columns='Age', inplace=True)

In [163]:
train_df['Sex'].replace(['male','female'],[0,1],inplace=True)
test_df['Sex'].replace(['male','female'],[0,1],inplace=True)

In [164]:
train_df.drop(columns=['Name', 'PassengerId', 'SibSp', 'Parch', 'Ticket', 'Cabin',
               'Embarked'], inplace = True)
test_df.drop(columns=['Name','PassengerId', 'SibSp', 'Parch', 'Ticket', 'Cabin',
              'Embarked'], inplace = True)

In [165]:
X = train_df.drop(columns='Survived')
y = train_df['Survived']
X_test = test_df.copy()

In [166]:
X_test

,Pclass,Sex,Family_Size,Family_Survival,FareBin_Code,AgeBin_Code
0,3,0,0,0.5,0,2
1,3,1,1,0.5,0,3
2,2,0,0,0.5,1,3
3,3,0,0,0.5,1,1
4,3,1,2,1.0,2,0
...,...,...,...,...,...,...
413,3,0,0,0.5,1,1
414,1,1,0,1.0,4,3
415,3,0,0,0.5,0,3
416,3,0,0,0.5,1,1


In [167]:
pd.DataFrame(X).head()

,Pclass,Sex,Family_Size,Family_Survival,FareBin_Code,AgeBin_Code
0,3,0,1,0.5,0,0
1,1,1,1,0.5,4,3
2,3,1,0,0.5,1,1
3,1,1,1,0.0,4,2
4,3,0,0,0.5,1,2


In [168]:
std_scaler = StandardScaler()
X = std_scaler.fit_transform(X)
X_test = std_scaler.transform(X_test)

In [169]:
pd.DataFrame(X).head()

,0,1,2,3,4,5
0,0.827377,-0.737695,0.059160,-0.060661,-1.407144,-1.143781
1,-1.566107,1.355574,0.059160,-0.060661,1.429429,1.540439
2,0.827377,1.355574,-0.560975,-0.060661,-0.698001,-0.249041
3,-1.566107,1.355574,0.059160,-1.604923,1.429429,0.645699
4,0.827377,-0.737695,-0.560975,-0.060661,-0.698001,0.645699


In [170]:
n_neighbors = [6,7,8,9,10,11,12,14,16,18,20,22]
algorithm = ['auto']
weights = ['uniform', 'distance']
leaf_size = list(range(1,50,5))

hyperparams = {'algorithm': algorithm, 'weights': weights, 'leaf_size': leaf_size, 
               'n_neighbors': n_neighbors}

gd=GridSearchCV(estimator = KNeighborsClassifier(), param_grid = hyperparams, verbose=True, 
                cv=10, scoring = "roc_auc")

gd.fit(X, y)
print(gd.best_score_)
print(gd.best_params_)

Fitting 10 folds for each of 240 candidates, totalling 2400 fits
0.8785236963472258
{'algorithm': 'auto', 'leaf_size': 6, 'n_neighbors': 14, 'weights': 'uniform'}


In [171]:
from sklearn.svm import SVC
from sklearn.model_selection import KFold
SVMC = SVC(probability=True)
kfold = KFold(n_splits=4)
Cs = [0.001, 0.01, 0.1, 1, 10, 50, 100,200,300, 1000, 3000]
gammas = [0.00001, 0.0001, 0.001, 0.01, 0.1, 1 ]
    
svc_param_grid = {'kernel': ['rbf'], 
                  'gamma': gammas,
                  'C': Cs}

gsSVMC = GridSearchCV(SVMC,param_grid = svc_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

gsSVMC.fit(X,y)

SVMC_best = gsSVMC.best_estimator_

print(SVMC_best.get_params())


# Best score
gsSVMC.best_score_

Fitting 4 folds for each of 66 candidates, totalling 264 fits
{'C': 1, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.1, 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}


0.8496091382862684

In [172]:
knn = KNeighborsClassifier(algorithm='auto', leaf_size=16, metric='minkowski', 
                           metric_params=None, n_jobs=1, n_neighbors=6, p=2, 
                           weights='uniform')
knn.fit(X, y)
pred1 = knn.predict(X_test)

In [173]:
# model = SVC(C=1000, break_ties=False,cache_size=200, class_weight=None, coef0=0.0, decision_function_shape= 'ovr', degree= 3, gamma=0.001, kernel= 'rbf', max_iter= -1, probability= True, random_state= None,shrinking= True, tol=0.001, verbose=False)
# model.fit(X,y)
# pred = model.predict(X_test)

In [174]:
# output = pd.DataFrame(pd.read_csv("../data/test.csv")['PassengerId'])
# output['Survived'] = y_pred1
# output.to_csv("../result/81.8.csv", index = False)

In [175]:
submission1 = pd.read_csv('../답/submission (1).csv')
del submission1['PassengerId']
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(pred1, submission1)
accuracy

0.8181818181818182